In [1]:
import gzip
import json
import gc
import math
import pandas as pd
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import random
from sklearn.model_selection import train_test_split
def jl_to_list(filename):
    output = []
    with gzip.open(filename, 'rb') as f:
        for line in f:
            output.append(json.loads(line))
    return output
samples = 50000
rows = jl_to_list('data/train_dataset.jl.gz')
if samples:
    rows = rows[:samples]
rows_train, rows_test = train_test_split(rows, test_size=0.2, random_state=42)
item_data = jl_to_list('data/item_data.jl.gz')
metadata = {x['item_id']:x for x in item_data}
all_items = list(metadata.keys())
y_true = [row['item_bought'] for row in rows_test]
test_dataset = jl_to_list('data/test_dataset.jl.gz')

In [2]:
class LastViewedBaseline(object):
    def __init__(self, all_items, fill_model = None, k = 10):
        self.all_items = all_items
        self.k = k
        self.fill_model = fill_model

    def fit(self, X=None, y=None):
        if self.fill_model is not None:
            self.fill_model.fit(X, y)
            
        return self
    
    def predict(self, X=None):
        y_pred = []
        for row in tqdm(X):
            recomendacion = self._predict_one(row)
            y_pred.append(recomendacion)
        return y_pred
    
    def _predict_one(self, row):
        viewed = [ev for ev in row['user_history'] if ev['event_type'] == 'view']
        viewed = sorted(viewed, key=lambda x: x['event_timestamp'], reverse=True)
        viewed = [ev['event_info'] for ev in viewed]

        recomendacion = []

        for item in viewed:
            if item not in recomendacion:
                recomendacion.append(item)
            if len(recomendacion) == self.k:
                return recomendacion
        
        missing_items = self.k - len(recomendacion)
        if self.fill_model is None:
            fill_items = random.choices(self.all_items, k=missing_items)
        else:
            fill_items = self.fill_model.predict([row])[0][:missing_items]

        return recomendacion + fill_items

In [3]:
class TopViewedItemsByMostFrequentDomainBaseline(object):
    def __init__(self, all_items, metadata, k=10, max_views=15, verbose='debug'):
        self.all_items = all_items
        self.metadata = metadata
        self.k = k
        self.max_views = max_views
        self.verbose = verbose
    
    def fit(self, X=None, y=None):
        self.sold_items_by_domain = defaultdict(lambda: defaultdict(int))

        for row in tqdm(X):
            viewed = [ev['event_info'] for ev in row['user_history'] if ev['event_type'] == 'view']
            for item in viewed:
                domain = metadata[item]['domain_id']
                self.sold_items_by_domain[domain][item] += 1
        return self
    
    def predict(self, X=None):
        y_pred = []
        if self.verbose == 'debug':
            for row in tqdm(X):
                recomendacion = self._predict_one(row)
                y_pred.append(recomendacion)
        else:
            for row in X:
                recomendacion = self._predict_one(row)
                y_pred.append(recomendacion)
        return y_pred
    
    def _predict_one(self, row):
        viewed = [ev['event_info'] for ev in row['user_history'] if ev['event_type'] == 'view']
        if len(viewed) == 0:
            return random.choices(self.all_items, k=self.k)
        domain = self.__visited_domains(row)
        domain = domain.most_common(1)[0][0]
        return self.__top_items(domain)
    
    def __visited_domains(self, row):
        domains = Counter()
        viewed = [ev['event_info'] for ev in row['user_history'] if ev['event_type'] == 'view']
        if len(viewed) > self.max_views:
            viewed = viewed[:self.max_views]
        for item in viewed:
            domain = self.metadata[item]['domain_id']
            domains[domain] += 1
        return domains
    
    def __top_items(self, domain):
        top = self.sold_items_by_domain[domain]
        top = Counter(top)
        top = top.most_common(self.k)
        recomendacion = [x[0] for x in top]
        missing_items = self.k - len(recomendacion)
        fill_items = random.choices(self.all_items, k=missing_items)
        
        return recomendacion + fill_items

In [25]:
fill_model = TopViewedItemsByMostFrequentDomainBaseline(all_items, metadata, verbose='none')
baseline = LastViewedBaseline(all_items, fill_model=fill_model)
baseline.fit(rows_train)
y_pred = baseline.predict(rows_test)
y_true = [x['item_bought'] for x in rows_test]

# y_pred = baseline.predict(test_dataset)

In [62]:
import statistics
import numpy as np

def ndcg(y_pred, y_true, metadata):
    ndcg_values = []
    
    y_pred_cleaned = clean(y_pred)
    
    for i in range(len(y_pred_cleaned)):
        dcg_value = dcg(y_pred_cleaned[i], y_true[i], metadata)
        idcg_value = idcg(y_pred_cleaned[i], y_true[i])
        ndcg_values.append(dcg_value / idcg_value)
    
    return statistics.mean(ndcg_values)

def clean(y_pred):
    results = []
    
    for items in y_pred:
        seen = set()
        seen_add = seen.add
        items_clean = [x for x in items if not (x in seen or seen_add(x))]
        missing_values = 10 - len(items_clean)
        items_clean = items_clean + [0] * missing_values
        results.append(items_clean)
    
    return results
        
def dcg(y_pred, y_true, metadata):
    values = []
    for i in range(len(y_pred)):
        rel_value = rel(y_pred[i], y_true, metadata)
        dcg_value = rel_value / np.log(i + 2.0)
        values.append(dcg_value)
    return np.sum(values)
    
def idcg(y_pred, y_true):
    return 22.42461597

def rel(y_hat, y, metadata):
    if y_hat == y:
        return 12.0
    if domain(y_hat, metadata) == domain(y, metadata):
        return 1.0
    return 0.0

def domain(item, metadata):
    return metadata[item]['domain_id']

In [63]:
ndcg(y_pred, y_true, metadata)

0.24676682960311602

In [ ]:
df = pd.DataFrame(data=y_pred)
df.to_csv("./results/fill_model.csv",sep=',',index=False,header=False)

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
baseline = TopViewedItemsByMostFrequentDomainBaseline(all_items, metadata)
baseline.fit(rows_train)
y_pred = baseline.predict(test_dataset)

In [ ]:
df = pd.DataFrame(data=y_pred)
df[df.isnull().any(axis=1)]